A ReACT agent that asks for movie and then tells you who is the oldest living cast member of that movie.

In [54]:
import os
from langchain_google_genai import  ChatGoogleGenerativeAI

In [55]:
# Langsmith variables
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="react_agent"

# Google Gemini API key
os.environ["GOOGLE_API_KEY"] = "__GOOGLE_KEY__"

# Tavily API key
os.environ["TAVILY_API_KEY"]="__TAVILY_KEY__"

In [56]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_tool = WikipediaQueryRun(description="This is the wiki tool. Use this tool to find out information about the movie cast member (date of birth, are they alive or not, date of death)", api_wrapper=wiki_api_wrapper)


In [57]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=1, 
    include_answer=True, 
    search_depth="advanced",
    description="This is the cast search tool. Use this tool to identify all the casts of the movie requested by the user."
)

tools = [search_tool, wiki_tool]

In [58]:
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature= 0.0)
llm_with_tools = llm.bind_tools(tools)
system_message = SystemMessage("""
        You are a ReACT agent that let's users know for requested movie, what are all the cast members,
        and who is the oldest of the cast. Use the following process to answer user query:
            Thought 1: User has asked me to find oldest cast member of the movie. I need to first find all the cast members of this movie.
            Action 1: search_tool cast of [movie_name]
            Observation 1: Have I got all the cast members of the movie?

            Thought 2: Now that I have the cast members, I need to find out the age of each of these cast members
            Action 2: wiki_tool [cast_member]
            Observation 2: have I got the age of all cast members, if not, keep iterating through the cast members and get their age.

            Thought 3: now that I have all the cast members and their age with me, I need to find out who is the oldest
            Action 3:  identify the highest age amongst these cast members
            Final Response: respond to user with the name of the oldest cast member 
    """)

messages = [system_message]

In [62]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools, state_modifier=system_message)
response = agent.invoke({"messages":HumanMessage("Midsommar")})
print(response.content)
print("========")
print("Full API response")
print("========")
print(response)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).